In [1]:
import pandas as pd
import gensim
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
import numpy as np
import nltk
import random
import logging

In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
from nltk.corpus import stopwords
# nltk.download("stopwords")
# nltk.download('punkt')
stop = set(stopwords.words('english'))

In [23]:
df = pd.read_csv('train.csv')
df['index1'] = df.index
df = df.fillna("")
df.shape

(404290, 7)

In [24]:
df.question2.isnull().value_counts()

False    404290
Name: question2, dtype: int64

In [25]:
# df = df[:10000]

In [26]:
def get_words(question) :
    return list(filter(lambda word: word not in stop and word.isalpha(), nltk.word_tokenize(question.lower())))

In [27]:
getLabeledSentence = lambda given_tuple: [
             LabeledSentence(get_words(given_tuple[0]),["question1_"+str(given_tuple[2])]),  
             LabeledSentence(get_words(given_tuple[1]),["question2_"+str(given_tuple[2])])]

In [28]:
flatten = lambda l: [item for sublist in l for item in sublist]

sentences = flatten(list(map(lambda x: getLabeledSentence(x) , df[['question1', 'question2', 'index1']].values)))

In [29]:
len(sentences)

808580

In [30]:
model = Doc2Vec(min_count=1, window=10, size=1000, sample=1e-4, negative=5, workers=8)

In [ ]:
model.build_vocab(sentences)

2017-04-20 11:11:41,458 : INFO : collecting all words and their counts
2017-04-20 11:11:41,460 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2017-04-20 11:11:41,578 : INFO : PROGRESS: at example #10000, processed 52633 words (448677/s), 9639 word types, 10000 tags
2017-04-20 11:11:41,642 : INFO : PROGRESS: at example #20000, processed 104890 words (835210/s), 13939 word types, 20000 tags
2017-04-20 11:11:41,700 : INFO : PROGRESS: at example #30000, processed 157293 words (924400/s), 17137 word types, 30000 tags
2017-04-20 11:11:41,758 : INFO : PROGRESS: at example #40000, processed 209517 words (905669/s), 19663 word types, 40000 tags
2017-04-20 11:11:41,816 : INFO : PROGRESS: at example #50000, processed 262203 words (919581/s), 21958 word types, 50000 tags
2017-04-20 11:11:41,875 : INFO : PROGRESS: at example #60000, processed 314424 words (904795/s), 23983 word types, 60000 tags
2017-04-20 11:11:41,933 : INFO : PROGRESS: at example #70000, processed

In [13]:
for epoch in range(10):
    random.shuffle(sentences)
    model.train(sentences)

2017-04-20 10:54:50,984 : INFO : training model with 8 workers on 13939 vocabulary and 1000 features, using sg=0 hs=0 sample=0.0001 negative=5 window=10
2017-04-20 10:54:50,985 : INFO : expecting 20000 sentences, matching count from corpus used for vocabulary survey
2017-04-20 10:54:52,342 : INFO : PROGRESS: at 1.88% examples, 6759 words/s, in_qsize 16, out_qsize 0
2017-04-20 10:54:53,604 : INFO : PROGRESS: at 17.14% examples, 31352 words/s, in_qsize 15, out_qsize 0
2017-04-20 10:54:54,845 : INFO : PROGRESS: at 32.39% examples, 40179 words/s, in_qsize 15, out_qsize 0
2017-04-20 10:54:57,723 : INFO : PROGRESS: at 47.60% examples, 33773 words/s, in_qsize 15, out_qsize 0
2017-04-20 10:55:00,162 : INFO : PROGRESS: at 62.91% examples, 32710 words/s, in_qsize 16, out_qsize 0
2017-04-20 10:55:02,646 : INFO : PROGRESS: at 78.15% examples, 31974 words/s, in_qsize 12, out_qsize 0
2017-04-20 10:55:03,131 : INFO : worker thread finished; awaiting finish of 7 more threads
2017-04-20 10:55:03,187 : 

In [14]:
xaxis1 = []
xaxis2 = []
yaxis = []
for i in range(0, 10000):
    x = model.docvecs['question1_'+str(i)]
    y = model.docvecs['question2_'+str(i)]
    xaxis1.append(np.dot(x,y))
    xaxis2.append(df.is_duplicate[i])
    yaxis.append(i)

In [15]:
import matplotlib.pyplot as plt

In [16]:
plt.plot(yaxis, xaxis1, "r", yaxis, xaxis2, "g")
# plt.show()

In [17]:
from sklearn.neural_network import MLPClassifier

In [18]:
clfNN = MLPClassifier(solver='lbfgs', alpha=1e-5,hidden_layer_sizes=(5, 2), random_state=1)

In [19]:
train_set = np.array(xaxis1[:9000]).reshape(-1,1)
train_labels = np.array(xaxis2[:9000]).reshape(-1,1)
print(train_set.shape)
print(train_labels.shape)

(9000, 1)
(9000, 1)


In [20]:
clfNN.fit(train_set, train_labels)

/home/indix/anaconda3/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:904: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MLPClassifier(activation='relu', alpha=1e-05, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(5, 2), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=1, shuffle=True,
       solver='lbfgs', tol=0.0001, validation_fraction=0.1, verbose=False,
       warm_start=False)

In [21]:
test_set = np.array(xaxis1[:9000]).reshape(-1,1)
test_labels = np.array(xaxis2[:9000]).reshape(-1,1)
print(test_set.shape)
print(test_labels.shape)

(9000, 1)
(9000, 1)


In [22]:
clfNN.score(test_set, test_labels)

0.62966666666666671